In [45]:
import csv
import os
import re
import sys
import libertyParser
from liberty.parser import parse_liberty
from iteration_utilities import deepflatten
def flatten(nested):
	for sublist in nested:
		if isinstance(sublist,str):
			return sublist
		else:
			for element in flatten(sublist):
				return element

def main():
    with open('./train_data/train_tt1p0v25c.csv', 'w', newline='') as csvfile:
        
        liberty_file = '/root/notebooks/EDA/train/lib1_tt1p0v25c_base_400.tlib'
        writer = csv.writer(csvfile)
        
        ### columns_names
        COLUMN_NAMES = ['temperature', 'voltage', 'capacitance', 'rise_capacitance', 'fall_capacitance', 
                'cell_rise', 'rise_transition', 'cell_fall', 'fall_transition', 'rise_power', 'fall_power',
                'index1_1', 'index1_2', 'index1_3', 'index1_4', 'index1_5', 'index1_6', 'index1_7',
                'index2_1', 'index2_2', 'index2_3', 'index2_4', 'index2_5', 'index2_6', 'index2_7']
        for i in range(1,50):
            string = 'value_'+str(i)
            COLUMN_NAMES.append(string)   
        writer.writerow(COLUMN_NAMES)
        
        ### Read File
        library = parse_liberty(open(liberty_file).read())
        operating_conditions = library.get_groups('operating_conditions')
        print(">>> process :",operating_conditions[0]['process'])
        print(">>> temperature :",operating_conditions[0]['temperature'])
        print(">>> voltage :",operating_conditions[0]['voltage'])
        print("\n\n")

        temperature = operating_conditions[0]['temperature']
        voltage = operating_conditions[0]['voltage']

        cells = library.get_groups('cell')
        cell_list = []
        for cell in cells:
            cell_list.append(cell.args[0])
        print("Cell List :",cell_list)
        for cellname in cell_list:
            # temperature, voltage, capacitance, rise_capacitance, fall_capacitance,
            # cell_rise, rise_transition, cell_fall, fall_transition, rise_power, fall_power,
            # index1 , index2
            # value
            feature = []
            pin_input = []
            cr_index1 = []
            cr_index2 = []
            cr_label = []
            rt_index1 = []
            rt_index2 = []
            rt_label = []
            cf_index1 = []
            cf_index2 = []
            cf_label = []
            ft_index1 = []
            ft_index2 = []
            ft_label = []
            rp_index1 = []
            rp_index2 = []
            rp_label = []
            fp_index1 = []
            fp_index2 = []
            fp_label = []

            rbcf_index1 = []
            rbcf_index2 = []
            rbcf_label = []

            rbft_index1 = []
            rbft_index2 = []
            rbft_label = []

            sbcr_index1 = []
            sbcr_index2 = []
            sbcr_label = []

            sbrt_index1 = []
            sbrt_index2 = []
            sbrt_label = []



            cell = library.get_group('cell', cellname)
            print("Cell :{} ".format(cellname) )
            #writer.writerows(cellname)
            #print(cell)
            pins = cell.get_groups('pin')
            for pin in pins:
                if pin.attributes['direction'] == 'input':
                    print("Pin: ",pin.args)
                    #writer.writerows(pin.args)
                    print(">>> direction:",pin.attributes['direction'])
                    print(">>> capacitance:",pin.attributes['capacitance'])
                    print(">>> rise_capacitance:",pin.attributes['rise_capacitance'])
                    print(">>> fall_capacitance:",pin.attributes['fall_capacitance'])
                    print("\n")
                    pin_input += [[pin.args,pin.attributes['capacitance'],pin.attributes['rise_capacitance'],pin.attributes['fall_capacitance']]]
                elif pin.attributes['direction'] == 'output':
                    print("Pin: ",pin.args)
                    print(">>> direction:",pin.attributes['direction'])
                    print(">>> function:",pin.attributes['function'])

                    timing = pin.get_groups('timing')
                    for timing_c in timing:
                        if timing_c.attributes['related_pin'] == 'RB':
                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_fall_index1 = timing_c.get_group('cell_fall').get_array('index_1')
                            cell_fall_index2 = timing_c.get_group('cell_fall').get_array('index_2')
                            cell_fall_value = timing_c.get_group('cell_fall').get_array('values')
                            #print(">>> cell_fall")
                            #print(">>> cell_fall_index1\n",cell_fall_index1)
                            #print(">>> cell_fall_index2\n",cell_fall_index2)
                            #print(">>> cell_fall_value\n",cell_fall_value)
                            rbcf_index1 += [cell_fall_index1]
                            rbcf_index2 += [cell_fall_index2]
                            rbcf_label += [cell_fall_value]

                            fall_transition_index1 = timing_c.get_group('fall_transition').get_array('index_1')
                            fall_transition_index2 = timing_c.get_group('fall_transition').get_array('index_2')
                            fall_transition_value = timing_c.get_group('fall_transition').get_array('values')
                            #print(">>> fall_transition")
                            #print(">>> fall_transition_index1\n",fall_transition_index1)
                            #print(">>> fall_transition_index2\n",fall_transition_index2)
                            #print(">>> fall_transition_value\n",fall_transition_value)
                            rbft_index1 += [fall_transition_index1]
                            rbft_index2 += [fall_transition_index2]
                            rbft_label += [fall_transition_value]
                        elif timing_c.attributes['related_pin'] == 'SB':
                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_rise_index1 = timing_c.get_group('cell_rise').get_array('index_1')
                            cell_rise_index2 = timing_c.get_group('cell_rise').get_array('index_2')
                            cell_rise_value = timing_c.get_group('cell_rise').get_array('values')
                            #print(">>> array_cell_rise")
                            #print(">>> array_cell_rise_index1\n",cell_rise_index1)
                            #print(">>> array_cell_rise_index2\n",cell_rise_index2)
                            #print(">>> array_cell_rise_value\n",cell_rise_value)
                            sbcr_index1 += [cell_rise_index1]
                            sbcr_index2 += [cell_rise_index2]
                            sbcr_label += [cell_rise_value]


                            rise_transition_index1 = timing_c.get_group('rise_transition').get_array('index_1')
                            rise_transition_index2 = timing_c.get_group('rise_transition').get_array('index_2')
                            rise_transition_value = timing_c.get_group('rise_transition').get_array('values')
                            #print(">>> rise_transition")
                            #print(">>> rise_transition_index1\n",rise_transition_index1)
                            #print(">>> rise_transition_index2\n",rise_transition_index2)
                            #print(">>> rise_transition_value\n",rise_transition_value)
                            sbrt_index1 += [rise_transition_index1]
                            sbrt_index2 += [rise_transition_index2]
                            sbrt_label += [rise_transition_value]
                        else : 

                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_rise_index1 = timing_c.get_group('cell_rise').get_array('index_1')
                            cell_rise_index2 = timing_c.get_group('cell_rise').get_array('index_2')
                            cell_rise_value = timing_c.get_group('cell_rise').get_array('values')
                            #print(">>> array_cell_rise")
                            #print(">>> array_cell_rise_index1\n",cell_rise_index1)
                            #print(">>> array_cell_rise_index2\n",cell_rise_index2)
                            #print(">>> array_cell_rise_value\n",cell_rise_value)
                            cr_index1 += [cell_rise_index1]
                            cr_index2 += [cell_rise_index2]
                            cr_label += [cell_rise_value]

                            rise_transition_index1 = timing_c.get_group('rise_transition').get_array('index_1')
                            rise_transition_index2 = timing_c.get_group('rise_transition').get_array('index_2')
                            rise_transition_value = timing_c.get_group('rise_transition').get_array('values')
                            #print(">>> rise_transition")
                            #print(">>> rise_transition_index1\n",rise_transition_index1)
                            #print(">>> rise_transition_index2\n",rise_transition_index2)
                            #print(">>> rise_transition_value\n",rise_transition_value)
                            rt_index1 += [rise_transition_index1]
                            rt_index2 += [rise_transition_index2]
                            rt_label += [rise_transition_value]

                            cell_fall_index1 = timing_c.get_group('cell_fall').get_array('index_1')
                            cell_fall_index2 = timing_c.get_group('cell_fall').get_array('index_2')
                            cell_fall_value = timing_c.get_group('cell_fall').get_array('values')
                            #print(">>> cell_fall")
                            #print(">>> cell_fall_index1\n",cell_fall_index1)
                            #print(">>> cell_fall_index2\n",cell_fall_index2)
                            #print(">>> cell_fall_value\n",cell_fall_value)
                            cf_index1 += [cell_fall_index1]
                            cf_index2 += [cell_fall_index2]
                            cf_label += [cell_fall_value]

                            fall_transition_index1 = timing_c.get_group('fall_transition').get_array('index_1')
                            fall_transition_index2 = timing_c.get_group('fall_transition').get_array('index_2')
                            fall_transition_value = timing_c.get_group('fall_transition').get_array('values')
                            #print(">>> fall_transition")
                            #print(">>> fall_transition_index1\n",fall_transition_index1)
                            #print(">>> fall_transition_index2\n",fall_transition_index2)
                            #print(">>> fall_transition_value\n",fall_transition_value)
                            ft_index1 += [fall_transition_index1]
                            ft_index2 += [fall_transition_index2]
                            ft_label += [fall_transition_value]



                    internal_power = pin.get_groups('internal_power')
                    for ip in internal_power:
                        #print("related_pin: ",ip.attributes['related_pin'])
                        rise_power_index1 = ip.get_group('rise_power').get_array('index_1')
                        rise_power_index2 = ip.get_group('rise_power').get_array('index_2')
                        rise_power_value = ip.get_group('rise_power').get_array('values')
                        #print(">>> rise_power")
                        #print(">>> rise_power_index1\n",rise_power_index1)
                        #print(">>> rise_power_index2\n",rise_power_index2)
                        #print(">>> rise_power_value\n",rise_power_value)
                        rp_index1 += [[rise_power_index1]]
                        rp_index2 += [[rise_power_index2]]
                        rp_label += [[rise_power_value]]

                        fall_power_index1 = ip.get_group('fall_power').get_array('index_1')
                        fall_power_index2 = ip.get_group('fall_power').get_array('index_2')
                        fall_power_value = ip.get_group('fall_power').get_array('values')
                        #print(">>> fall_power")
                        #print(">>> fall_power_index1\n",fall_power_index1)
                        #print(">>> fall_power_index2\n",fall_power_index2)
                        #print(">>> fall_power_value\n",fall_power_value)
                        fp_index1 += [[fall_power_index1]]
                        fp_index2 += [[fall_power_index2]]
                        fp_label += [[fall_power_value]]

                    print("\n")

            #print(pin_input)
            cr = [1,0,0,0,0,0]
            rt = [0,1,0,0,0,0]
            cf = [0,0,1,0,0,0]
            ft = [0,0,0,1,0,0]
            for i in range(len(timing)):
                for j in range(len(pin_input)):
                    if [timing[i].attributes['related_pin']] == pin_input[j][0]:
                        crf = pin_input[j][1:]
                wr1 = ''
                wr2 = ''
                wr3 = ''
                wr4 = ''
                if timing[i].attributes['related_pin'] == 'RB' :
                    crf = [0.,0.,0.]
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cf")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[rbcf_index1[0]]+[rbcf_index2[0]]+[rbcf_label[0]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[rbcf_index1[0]]+[rbcf_index2[0]]+[rbcf_label[0]]))
                    #print("ft")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[rbft_index1[0]]+[rbft_index2[0]]+[rbft_label[0]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[rbft_index1[0]]+[rbft_index2[0]]+[rbft_label[0]]))
                elif timing[i].attributes['related_pin'] == 'SB' :
                    crf = [0.,0.,0.]
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cr")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[sbcr_index1[0]]+[sbcr_index2[0]]+[sbcr_label[0]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[sbcr_index1[0]]+[sbcr_index2[0]]+[sbcr_label[0]]))
                    #print("rt")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[sbrt_index1[0]]+[sbrt_index2[0]]+[sbrt_label[0]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[sbrt_index1[0]]+[sbrt_index2[0]]+[sbrt_label[0]]))
                
                else :  
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cr")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[cr_index1[i]]+[cr_index2[i]]+[cr_label[i]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[cr_index1[i]]+[cr_index2[i]]+[cr_label[i]]))
                    
                    #print("rt")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[rt_index1[i]]+[rt_index2[i]]+[rt_label[i]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[rt_index1[i]]+[rt_index2[i]]+[rt_label[i]])) 
                    
                    #print("cf")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[cf_index1[i]]+[cf_index2[i]]+[cf_label[i]]))))
                    wr3 = list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[cf_index1[i]]+[cf_index2[i]]+[cf_label[i]]))
                    
                    #print("ft")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[ft_index1[i]]+[ft_index2[i]]+[ft_label[i]]))))
                    wr4 = list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[ft_index1[i]]+[ft_index2[i]]+[ft_label[i]]))
                    
                    writer.writerow(wr1)
                    writer.writerow(wr2)
                    writer.writerow(wr3)
                    writer.writerow(wr4)
                    
            for i in range(len(internal_power)):
                for j in range(len(pin_input)):
                    if [internal_power[i].attributes['related_pin']] == pin_input[j][0]:
                        crf = pin_input[j][1:]
                wr5 = ''
                wr6 = ''
                #print("power pin:",internal_power[i].attributes['related_pin'] )
                #print("rp")
                rp = [0,0,0,0,1,0]
                #print((list(deepflatten([temperature]+[voltage]+[crf]+[rp]+[rp_index1[i]]+[rp_index2[i]]+[rp_label[i]]))))
                wr5 = list(deepflatten([temperature]+[voltage]+[crf]+[rp]+[rp_index1[i]]+[rp_index2[i]]+[rp_label[i]]))
                
                #print("fp")
                fp = [0,0,0,0,0,1]
                #print((list(deepflatten([temperature]+[voltage]+[crf]+[fp]+[fp_index1[i]]+[fp_index2[i]]+[fp_label[i]]))))
                wr6 = list(deepflatten([temperature]+[voltage]+[crf]+[fp]+[fp_index1[i]]+[fp_index2[i]]+[fp_label[i]]))
            
                writer.writerow(wr5)
                writer.writerow(wr6)
            #break
            
    #writer.close()
    
if __name__ == '__main__':
    main()

>>> process : 1.0
>>> temperature : 25.0
>>> voltage : 1.0



Cell List : ['AN2AM12', 'AN2AM8', 'AN2M12', 'AN2M2', 'AN2M4', 'AN2M6', 'AN2M8', 'AN3M12', 'AN3M2', 'AN3M4', 'AN3M6', 'AN3M8', 'AN3YM12', 'AN3YM16', 'AN3YM22', 'AN3YM6', 'AN3YM8', 'AN4M12', 'AN4M4', 'AN4M6', 'AN4M8', 'AN4YM12', 'AN4YM16', 'AN4YM6', 'AN4YM8', 'AO211M1', 'AO211M8', 'AO211YM6', 'AO211YM8', 'AO21M12', 'AO21M6', 'AO21M8', 'AO221M1', 'AO221M8', 'AO222M1', 'AO22B10M1', 'AO22B10M2', 'AO22B10M4', 'AO22B10M8', 'AO22B11M1', 'AO22B11M2', 'AO22B11M4', 'AO22B11M8', 'AO22M12', 'AO22M6', 'AO22M8', 'AO31M1', 'AO32M1', 'AO33M1', 'AOI211M1', 'AOI211M8', 'AOI21B01M1', 'AOI21B01M12', 'AOI21B01M16', 'AOI21B01M2', 'AOI21B01M4', 'AOI21B01M6', 'AOI21B01M8', 'AOI21B10M1', 'AOI21B10M12', 'AOI21B10M16', 'AOI21B10M2', 'AOI21B10M4', 'AOI21B10M6', 'AOI21B10M8', 'AOI21B20M1', 'AOI21B20M2', 'AOI21B20M4', 'AOI21B20M8', 'AOI21M12', 'AOI21M3', 'AOI21M4', 'AOI21M6', 'AOI21M8', 'AOI221M1', 'AOI221M2', 'AOI221M4', 'AOI221M6', 'AOI221M8', 'AOI222M1

In [46]:

print('1111')

1111
